<h1 style="text-align: center; color: blue; font-weight: bold; font-size: 30px;"> 
   Automating Excel tasks with openpyxl <br>
   ......................................................................................................................

</h1>

#### Advantages :
Automating Excel tasks with openpyxl allows users to efficiently manage and manipulate Excel files without the need for manual intervention. This Python library enables automating repetitive tasks such as data entry, formatting, chart generation, and calculation updates. By leveraging openpyxl, users can programmatically create, modify, and read Excel workbooks, making it ideal for tasks like batch processing of reports, importing and exporting data between applications, or applying complex formulas across large datasets. This automation saves time, reduces errors, and enhances productivity by enabling users to handle large volumes of data seamlessly.

#### About Dataset:
The healthcare-stroke dataset contains information about patients who have had a stroke. It typically includes features such as age, gender, hypertension, heart disease, smoking status, work type, marital status, body mass index (BMI), glucose levels, and whether the patient has experienced a stroke. The dataset is used to analyze factors that contribute to stroke risk and can be used for predictive modeling to identify individuals at higher risk of having a stroke.

In [1]:
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill
from openpyxl.chart import BarChart, Reference
import pandas as pd


In [2]:
# Load the dataset into a pandas DataFrame
file_path = "healthcare-stroke-data.xlsx"  # Update with your file path
data = pd.read_excel(file_path)

In [23]:
data.head(3)

,gender,age,hypertension,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,Yes,Self-employed,Rural,202.21,32.5,never smoked,1
2,Male,80.0,0,Yes,Private,Rural,105.92,32.5,never smoked,1


In [3]:
# Load the workbook
wb = load_workbook(file_path)
sheet = wb.active


<h1 style="text-align: center; color: blue; font-weight: bold; font-size: 20px;">   1. Filter and Save High-Risk Stroke Patients </font>

In [5]:
high_risk_wb = Workbook()
high_risk_sheet = high_risk_wb.active
high_risk_sheet.title = "High Risk Patients"

In [6]:
# Copy headers
for col in sheet.iter_cols(1, sheet.max_column):
    high_risk_sheet.cell(row=1, column=col[0].column).value = col[0].value

In [7]:
# Filter data for stroke == 1
row_index = 2
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, values_only=True):
    if row[-1] == 1:  # Stroke column
        for col_index, value in enumerate(row, start=1):
            high_risk_sheet.cell(row=row_index, column=col_index).value = value
        row_index += 1

In [9]:
# Save the filtered high-risk patients
high_risk_wb.save("high_risk_patients.xlsx")

In [25]:
df=pd.read_excel('high_risk_patients.xlsx')
df.head(3)

,gender,age,hypertension,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,Yes,Self-employed,Rural,202.21,32.5,never smoked,1
2,Male,80.0,0,Yes,Private,Rural,105.92,32.5,never smoked,1


In [27]:
df.tail(3)

,gender,age,hypertension,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
246,Female,75.0,0,Yes,Self-employed,Rural,78.80,29.3,formerly smoked,1
247,Male,71.0,1,Yes,Self-employed,Rural,87.80,32.5,Unknown,1
248,Female,78.0,0,Yes,Private,Rural,78.81,19.6,Unknown,1


<font color=green> Here only the high risk patients ( ie; stroke value with only 1 ) are filtered out and saved.

<h1 style="text-align: center; color: blue; font-weight: bold; font-size: 20px;">2. Summary Report

In [30]:

summary_sheet = wb.create_sheet(title="Summary Report")
summary_sheet["A1"] = "Metric"
summary_sheet["B1"] = "Value"
summary_stats = {
    "Average Age": data["age"].mean(),
    "Average BMI": data["bmi"].mean(),
    "Average Glucose Level": data["avg_glucose_level"].mean(),
    "Total Patients": len(data),
    "Stroke Cases": data["stroke"].sum()
}
for idx, (metric, value) in enumerate(summary_stats.items(), start=2):
    summary_sheet[f"A{idx}"] = metric
    summary_sheet[f"B{idx}"] = round(value, 2)


<h1 style="text-align: center; color: blue; font-weight: bold; font-size: 20px;">  3. Categorical Data Count

In [36]:
category_sheet = wb.create_sheet(title="Category Counts")
category_sheet["A1"] = "Category Type"
category_sheet["B1"] = "Category"
category_sheet["C1"] = "Count"

# Flatten and write category counts
gender_counts = data["gender"].value_counts()
smoking_counts = data["smoking_status"].value_counts()
category_data = pd.concat([gender_counts, smoking_counts], keys=["Gender", "Smoking Status"])
category_data_flat = category_data.reset_index()
category_data_flat.columns = ["Category Type", "Category", "Count"]

for idx, row in category_data_flat.iterrows():
    category_sheet[f"A{idx+2}"] = row["Category Type"]
    category_sheet[f"B{idx+2}"] = row["Category"]
    category_sheet[f"C{idx+2}"] = row["Count"]

<h1 style="text-align: center; color: blue; font-weight: bold; font-size: 20px;">  4. Highlight Critical Cases

In [41]:
fill_red = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")

for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
    glucose = row[6].value  # avg_glucose_level
    bmi = row[7].value  # bmi
    if glucose > 200 or bmi > 30:
        for cell in row:
            cell.fill = fill_red

<h1 style="text-align: center; color: blue; font-weight: bold; font-size: 20px;">   5. Add a Chart (Stroke Count by Gender)

In [59]:

chart_sheet = wb.create_sheet(title="Charts")
chart_sheet.append(["Gender", "Stroke Count"])
stroke_counts = data[data["stroke"] == 1]["gender"].value_counts()

for gender, count in stroke_counts.items():
    chart_sheet.append([gender, count])

chart = BarChart()
data_ref = Reference(chart_sheet, min_col=2, min_row=1, max_col=2, max_row=len(stroke_counts) + 1)
categories_ref = Reference(chart_sheet, min_col=1, min_row=2, max_row=len(stroke_counts) + 1)
chart.add_data(data_ref, titles_from_data=True)
chart.set_categories(categories_ref)
chart.title = "Stroke Count by Gender"
chart.x_axis.title = "Gender"
chart.y_axis.title = "Count"

chart_sheet.add_chart(chart, "E5")
output_file = "automated_healthcare_analysis.xlsx"  # Update the path if needed
wb.save(output_file)
print(f"File saved successfully as {output_file}.")


File saved successfully as automated_healthcare_analysis.xlsx.
